In [43]:
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import copy

In [44]:
# Load pre-trained resnet101 model and modify it for 15 cat breeds
resnetModel = models.resnet101(pretrained=True)


In [45]:
# # Freeze all layers in the model
# for param in resnetModel.parameters():
#     param.requires_grad = False
num_features = resnetModel.fc.in_features
resnetModel.fc = nn.Linear(num_features, 15)

additional_layers = nn.Sequential(
    nn.Linear(num_features, 256),
    nn.BatchNorm1d(256),
    nn.LeakyReLU(negative_slope=0.01),
    nn.Dropout(0.7),
    nn.Dropout(0.7),

    nn.Linear(256, 256),
    nn.BatchNorm1d(256),
    nn.LeakyReLU(negative_slope=0.01),
    nn.Dropout(0.7),

    nn.Linear(256, 15),
    nn.Softmax(dim=1)  
)
# Attach the additional layers to the model
resnetModel.add_module("additional_layers", additional_layers)

# Unfreeze the newly added layers
# for param in resnetModel.fc.parameters():
#     param.requires_grad = True

# for param in resnetModel.additional_layers.parameters():
#     param.requires_grad = True
# # unfreeze every layer
# for param in resnetModel.parameters():
#     param.requires_grad = True

In [46]:
# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnetModel = resnetModel.to(device)


In [47]:

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnetModel.parameters(), lr=0.001, momentum=0.9)


In [48]:
# Data transformations and loaders
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_data = datasets.ImageFolder(root='./cats/Training', transform=transform)
val_data = datasets.ImageFolder(root='./cats/Validation', transform=transform)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

In [49]:
def train(model, criterion, optimizer, train_loader, device):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        
        # Forward
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Calculate average loss and accuracy
    avg_loss = running_loss / len(train_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

# Validation loop
def validate(model, criterion, val_loader, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return running_loss / len(val_loader), accuracy


In [50]:

# Assume all imports and model initialization are already done
num_epochs = 20
best_model_wts = copy.deepcopy(resnetModel.state_dict())
best_acc = 0.0

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print('-' * 10)

    # Train and validate
    train_loss, train_accuracy = train(resnetModel, criterion, optimizer, train_loader, device)
    val_loss, val_accuracy = validate(resnetModel, criterion, val_loader, device)

    print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')
    print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

    # Check if this is the best model based on validation accuracy
    if val_accuracy > best_acc:
        best_acc = val_accuracy
        best_model_wts = copy.deepcopy(resnetModel.state_dict())

    print()


Epoch 1/20
----------
Train Loss: 1.3863, Train Accuracy: 60.94%
Validation Loss: 0.6564, Validation Accuracy: 79.20%

Epoch 2/20
----------
Train Loss: 0.5628, Train Accuracy: 81.99%
Validation Loss: 0.5406, Validation Accuracy: 82.31%

Epoch 3/20
----------
Train Loss: 0.3596, Train Accuracy: 88.81%
Validation Loss: 0.5567, Validation Accuracy: 80.80%

Epoch 4/20
----------
Train Loss: 0.2131, Train Accuracy: 94.00%
Validation Loss: 0.5642, Validation Accuracy: 81.24%

Epoch 5/20
----------
Train Loss: 0.1133, Train Accuracy: 97.61%
Validation Loss: 0.5850, Validation Accuracy: 82.31%

Epoch 6/20
----------
Train Loss: 0.0618, Train Accuracy: 98.97%
Validation Loss: 0.6015, Validation Accuracy: 82.67%

Epoch 7/20
----------
Train Loss: 0.0357, Train Accuracy: 99.64%
Validation Loss: 0.6180, Validation Accuracy: 81.78%

Epoch 8/20
----------
Train Loss: 0.0216, Train Accuracy: 99.93%
Validation Loss: 0.6244, Validation Accuracy: 82.49%

Epoch 9/20
----------
Train Loss: 0.0180, Train 

In [ ]:

# Load best model weights
resnetModel.load_state_dict(best_model_wts)

# Save the best model
torch.save(resnetModel.state_dict(), './resnetModels/d.pth')
print('Best model saved with accuracy:', best_acc)
